## **Welcome to my IBM Data Science Capstone Project**

This project is based on NASA Meteorite Landings dataset, which you can find right here:           
https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh

Thanks for visiting and I hope you will find my project intresting :)

# Dataset Preparation

okey so we have download our Meteorite_Landings.csv, in this project I imported dataset into MySQL local host. if you want to repeat my steps remember to change ‘max_allowed_packet’ size, default setting is 1MB. To change our ‘max_allowed_packet’ we need to open XAMPP and in Config->my.ini change ‘max_allowed_packet’ to 50M (with mean 50 MB), restart MySQL and now you can import your data sat. 

### First of all import libraries that we are going to use

In [6]:
import mysql.connector
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 

import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
from folium import plugins


from geopy.geocoders import Nominatim
import pycountry_convert as pc

from pprint import pprint
from typing import Tuple 

import random
from time import sleep

##### Now lets connect our notebook with dataset

In [28]:
try:
    mydb = mysql.connector.connect(
        host = "localhost", ## local host name
        user = "root", ## root name
        passwd = "", ## password if you set one 
        database = "meteorite_landings" ## database name 
    )
    
    query  = "SELECT * FROM meteorite_landings;"
    df = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

df.head()

C:\Users\syzmon\AppData\Local\Temp\ipykernel_6820\3658188709.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,mydb)


,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,0,50.7750,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.1833,10.23330,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.2167,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.8833,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.1667,-64.95000,"(-33.16667, -64.95)"


In [29]:
## Drop rows with GeoLocation='(0.0, 0.0)'
df = df[df.GeoLocation != '(0.0, 0.0)']
df = df[df.year != 0]

# Change mass from gram into kilogram 
df['mass (g)'] = (df['mass (g)']/1000).round(2)

## Rename dataframe
df.rename(columns={'name':'NAME', 'id':'ID', 'nametype':'NAME_TYPE', 'recclass':'CLASS','mass (g)':'MASS [kg]', 'fall':'FALL', 'year':'YEAR', 'reclat':'LAT','reclong':'LONG', 'GeoLocation':'COORDINATE'}, inplace=True)
df.head()


,NAME,ID,NAME_TYPE,CLASS,MASS [kg],FALL,YEAR,LAT,LONG,COORDINATE
1,Aarhus,2,Valid,H6,0.72,Fell,1951,56.1833,10.2333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107.00,Fell,1952,54.2167,-113.0000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1.91,Fell,1976,16.8833,-99.9000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,0.78,Fell,1902,-33.1667,-64.9500,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4.24,Fell,1919,32.1000,71.8000,"(32.1, 71.8)"


In [30]:
df4stat = df
df4stat = df[df['MASS [kg]'] >= 2]
df4stat = df4stat.assign(COUNTRY='Nan', CONTINENT='Nan')
df4stat.shape

(2268, 12)

In [31]:
continent_dict = {
    "NA": "North America",
    "SA": "South America",
    "AS": "Asia",
    "AF": "Africa",
    "OC": "Oceania",
    "EU": "Europe",
    "AQ" : "Antarctica"
}

In [32]:
geolocator = Nominatim(user_agent='IBMDataScienceCapstone')

for idx, row in df4stat.iterrows():
    sleep(1)
    location = geolocator.reverse([row['LAT'], row['LONG']], exactly_one=True, language='en')

    if location is None:
        df4stat.loc[idx, 'COUNTRY'] = "Antarctica"
        df4stat.loc[idx, 'CONTINENT'] = "Antarctica"
    else:
        address = location.raw['address']
        df4stat.loc[idx, 'COUNTRY'] = address.get('country', '')
        
        continent_code = address.get('country_code', '').upper()
        df4stat.loc[idx, 'CONTINENT']  = continent_dict[pc.country_alpha2_to_continent_code(continent_code)] 
df4stat.head()

GeocoderServiceError: Non-successful status code 502

In [ ]:
df4stat.to_csv(r'C:/Users/syzmon/Desktop/IBM data science capstone/db.csv', index=False)

In [ ]:
# import time
# def chunker(seq, size):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))
#
# wait_time = 1
# time.sleep(random.randint(1*100,wait_time_batch*100)/100) 
# batch_number = len(df4stat.index)%2000
# rV = []
# for i in chunker(df4stat, batch_number):
#     geolocator = Nominatim(user_agent='trial'+str(random.randint(0,1000))) 
#     for idx, row in i.iterrows():   
#         location = geolocator.reverse([row['LAT'], row['LONG']], exactly_one=True)
#         if location is None:
#             i.loc[idx, 'COUNTRY'] = "Antarctica"
#             i.loc[idx, 'CONTINENT'] = "Antarctica"
#         else:
#             address = location.raw['address']
#             i.loc[idx, 'CONTINENT'] = address.get('country', '').upper()
#
#             continent_code = address.get('country_code', '').upper()
#             i.loc[idx, 'CONTINENT']  = continent_dict[pc.country_alpha2_to_continent_code(continent_code)] 
#             time.sleep(random.randint(1*100,wait_time*100)/100)     
#     rV.append(i)
# new_df = pd.DataFrame(np.reshape(rV,df4stat.shape))
# new_df.head()

In [ ]:
mas_max = df4stat['MASS [kg]'].max()

map = folium.Map(location=[40, 30], zoom_start=2)
for idx, row in df4stat.iterrows():
    html = '''
            <html>
                <body>
                    <div>
                        <p> Name:'''+ row['NAME'] +'''<br> Year:'''+ str(row['YEAR']) + '''<br> ID:'''+ str(row['ID']) + '''</p>
                    </div>
                </body>
            </html>  
            ''' 
    
    radius=(row['MASS [kg]']/mas_max)*100
    circle=folium.Circle([row['LAT'], row['LONG']], radius=radius, color='red', stroke=False, fill=True, fill_opacity=0.75).add_child(folium.Popup(html=html, max_width=800))
    map.add_child(circle)
    
minimap = plugins.MiniMap(toggle_display=True, width=300, height=100)
map.add_child(minimap)
map

### Now lets create dashboard

In [ ]:
import dash                                     # pip install dash
from dash import dcc, html, Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots

In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
df4dash = df4stat.groupby(['CONTINENT', 'YEAR'])['ID'].count().reset_index()
df4dash.rename(columns={'ID':'COUNT'}, inplace=True)
ds_min = df4dash['YEAR'].min()
ds_max = df4dash['YEAR'].max()
ds_median = df4dash.groupby(['YEAR']).median()
marks={
        i: {
            "label": f"{i}",
            "style": {"transform": "rotate(0deg)", "white-space": "nowrap"},
        }
        for i in range(ds_min, ds_max, 10)
}
df4dash.head()

C:\Users\syzmon\AppData\Local\Temp\ipykernel_13720\1007916315.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_median = df4dash.groupby(['YEAR']).median()


,CONTINENT,YEAR,COUNT
0,Africa,1903,4
1,Africa,1907,1
2,Africa,1909,1
3,Africa,1910,1
4,Africa,1911,1


In [ ]:
app.layout = html.Div([
    html.H1('Meteorite Landings from NASA dataset',
        style={'textAlign': 'center','color':'#503D36','font-size':75}
    ),
    html.Div([
        html.Div([
            dcc.Checklist(id='my-checklist', value=df4dash['CONTINENT'].unique(),
                            options=[{'label': i, 'value': i} for i in sorted(df4stat['CONTINENT'].unique())],
                            style={'font-size': '20px'},
                            labelClassName='mr-3',
                            inline=True
                        )
            ],style={'width':'50%', "margin": "15px"}),

        html.Div([
            html.Label("Select Year", htmlFor="years", style={'font-size': '20px', 'width':'50%'}),
            dcc.RangeSlider(id='sslider',
                            min=ds_min, max=ds_max, step=10, 
                            value=[df4stat['YEAR'].min(), df4stat['YEAR'].max()],
                            updatemode='drag',
                            tooltip={"placement": 'bottom', "always_visible": True},
                            marks=marks, 
                        ),
            ], style={'width':'50%'})
            
    ],style={'display':'flex'}),
    
    html.Div([
        dcc.Graph(id='my-hist', figure={}, style={'width':'50%'}),
        dcc.Graph(id="graph", figure={}, style={'width':'50%'})
    ],style={'display':'flex'}),
    
    dcc.Dropdown(id='dpdn2', value=['North America','Europe', 'Asia'], multi=True,
                 options=[{'label': x, 'value': x} for x in
                          df4stat['CONTINENT'].unique()]),
    html.Div([
        dcc.Graph(id='pie-graph', figure={},style={'width':'50%'}, className='six columns'),
        dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None, # I assigned None for tutorial purposes. By defualt, these are None, unless you specify otherwise.
                  config={
                      'staticPlot': False,     # False - Graph is static, so you can't for example save image
                      'scrollZoom': True,      # True - Allows you to zoom your graph
                      'doubleClick': 'reset',  # 'reset' - Reset image zoom on double click 
                      'showTips': False,       # Tips for users
                      'displayModeBar': True,  # Mode bar
                      'watermark': True,       # plotly watermark 
                      'modeBarButtonsToRemove': ['select2d'] # Remove mode bar features
                    },style={'width':'50%'},
                  className='six columns'
                  )
    ], style={'display':'flex'})
])

In [ ]:
@app.callback(
    Output(component_id='my-hist', component_property='figure'),
    [Input(component_id='my-checklist', component_property='value')]
)
def update_hist(continents):
    df_hist = df4stat.groupby(['CONTINENT'])['ID'].count().reset_index()
    df_hist = df_hist[df_hist['CONTINENT'].isin(continents)]
    print(continents)
    fig = px.bar(x=df_hist['CONTINENT'], y=df_hist['ID'], labels={'x':'Continent', 'y':'Count'})
    return fig

In [ ]:
@app.callback(
    Output(component_id='graph', component_property='figure'),
    [Input(component_id='sslider', component_property='value')]
)
def update_graph(years):
    dff = df4dash[df4dash['YEAR']>=years[0]]
    dff = dff[dff['YEAR']<=years[1]]
    dff = dff.groupby(['CONTINENT', 'YEAR'])['ID'].count().reset_index()


    dff = dff.pivot(index='CONTINENT', columns='YEAR')['ID'].fillna(0)
    fig=px.imshow(dff, x=dff.columns , y=dff.index)
    fig.update_layout(title=f"Heat map for years: {years}", xaxis_title='year', yaxis_title='cont')
    return (fig)

In [ ]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value'),
)
def update_graph(continent_chosen):
    dff = df4dash[df4dash.CONTINENT.isin(continent_chosen)]
    fig = px.scatter(data_frame=dff, x='YEAR', y='COUNT', color='CONTINENT')
    return fig

In [ ]:
# Dash version 1.16.0 or higher
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, continent_chosen):
    if hov_data is None:
        dff2 = df4dash[df4dash['YEAR'] == ds_min]
        dff2 = df4dash[df4dash.CONTINENT.isin(continent_chosen)]
        fig2 = px.pie(data_frame=dff2, values='COUNT', names='CONTINENT',
                      title=f'Year: {ds_min}')
        return fig2
    else:
        dff2 = df4dash[df4dash.CONTINENT.isin(continent_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2['YEAR'] == hov_year]
        fig2 = px.pie(data_frame=dff2, values='COUNT', names='CONTINENT', title=f'Year: {hov_year}')
        return fig2


In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
